In [7]:
import zipfile
import requests
import pandas as pd

### 설정값 입력

In [52]:
#파라미터 값을 입력한다.
api_key = "281f927788a0ae993bc626bc1b52ca02f51d619b" #DART에서 발급받은 API Key
bsns_year = "2021" #재무재표를 불러올 사업년도. 2015년 이후 부터 정보제공
reprt_code = "11011" #보고서 코드. 1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011
fs_div = "OFS" #개별/연결구분. CFS:연결재무제표, OFS:재무제표

#재무제표를 추출하고자 하는 기업의 고유번호를 리스트로 입력한다. (여러 기업의 정보를 한번에 추출해야 하므로)
#기업 고유번호는 아래 1번 코드를 실행하면 저장되는 CORPCODE.xlsx 파일을 열어 확인이 가능하다.
corp_codes = ["155319", "1089226", "100814", "124504"] #반드시 따옴표를 붙여 문자열로 입력할 것!

### 1. 회사 고유번호 불러와 엑셀파일(CORPCODE.xlsx)로 저장하기

In [48]:
#DART API를 이용해 기업고유번호 최신 데이터를 요청한다.
url = "https://opendart.fss.or.kr/api/corpCode.xml"
params = {
    "crtfc_key" : api_key
}
res = requests.get(url, params=params)

#기업고유번호 최신 데이터를 zip파일로 저장한다.
open("기업고유번호.zip", "wb").write(res.content)

#저장한 Zip 파일의 압축을 해제한다.
zip_ref = zipfile.ZipFile("기업고유번호.zip", "r")
zip_ref.extractall()

#압축 해제 후 나오는 CORPCODE.xml 파일을 판다스 DataFrame으로 변환한다.
df_corpcode = pd.read_xml("CORPCODE.xml")

#DataFrame을 엑셀로 저장한다.
df_corpcode.to_excel("CORPCODE.xlsx")

### 2. 기업별 재무제표 불러오기

In [54]:
#개별 회사의 재무제표 데이터(DataFrame)를 담을 빈 리스트를 만들어 놓는다.
result = []

#결과파일에 회사명을 넣어야 보기 편하므로 CORPCODE.xlsx 파일을 불러와 참조 테이블로 사용한다.
df_corpcode = pd.read_excel("CORPCODE.xlsx", index_col=0) #CORPCODE.xlsx 파일이 없으면 에러나니 사전에 체크 필요

#단일회사 전체 재무제표 API 주소를 입력한다.
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

#기업 고유번호 리스트에 있는 고유번호 값을 반복문을 돌면서 접근해 데이터를 긁어온다.
for idx, corp_code in enumerate(corp_codes):
    #기업 고유번호에 해당하는 회사명을 df_corpcode에서 참조해 불러온다.
    corp_name = df_corpcode.loc[(df_corpcode["corp_code"] == int(corp_code)), "corp_name"].values[0]

    #request에 사용하는 corp_code 파라미터 값은 8자리 숫자여야 한다. 8자리 숫자가 아닐경우 앞에 0을 붙여준다.
    corp_code_req = corp_code
    if len(corp_code_req) != 8:
        for _ in range(8 - len(corp_code_req)):
            corp_code_req = "0" + corp_code_req
    params = {
        "crtfc_key" : api_key,
        "corp_code" : corp_code_req,
        "bsns_year" : bsns_year,
        "reprt_code" : reprt_code,
        "fs_div" : fs_div
    }
    res = requests.get(url, params=params)
    data = res.json()

    #결과 로드시 에러가 발생할 수 있으므로 try문으로 에러있는 결과는 건너뛴다
    try:
        data["list"]
    except:
        try:
            error_msg = data["message"]
        except:
            error_msg = "불명"
        print("({}/{}) {} : 데이터 저장실패! 원인 : {}{}".format(idx+1, len(corp_codes), corp_name, error_msg, " "*50))
        continue

    df = pd.DataFrame(data["list"]) #재무제표 결과를 df로 변환한다.
    df["corp_name"] = corp_name #나중에 보기 편하게 df에 회사명을 입력해 준다.
    result.append(df) #result 변수(리스트)에 df를 하나씩 차곡차곡 추가한다.
    print("({}/{}) {} : 데이터 저장성공{}".format(idx+1, len(corp_codes), corp_name, " "*50))

#result 변수에 담긴 df들을 하나의 DataFrame 몽땅 뭉쳐 df_result_final 변수에 저장한다.
df_result_final = pd.concat(result, ignore_index=True)
df_result_final.to_excel("재무제표 크롤링 결과.xlsx", index=False)
print("\n크롤링이 완료되었습니다. '재무제표 크롤링 결과.xlsx' 파일로 결과가 저장되었습니다.")

(1/4) POSCO홀딩스 : 데이터 저장성공                                                  
(2/4) 포스코인재창조원 : 데이터 저장실패! 원인 : 조회된 데이타가 없습니다.                                                  
(3/4) 포스코건설 : 데이터 저장실패! 원인 : 조회된 데이타가 없습니다.                                                  
(4/4) 포스코인터내셔널 : 데이터 저장성공                                                  

크롤링이 완료되었습니다. '재무제표 크롤링 결과.xlsx' 파일로 결과가 저장되었습니다.
